# Generador de votos

Este simulador de votos lee la tabla de votos y emite un voto cada intervalo de tiempo. Además actualiza el número de votos pendientes que quedan por emitir para cada municipio y cada partido. Idealmente, los votos aparecerían en el stream sin intervalos fijos y podrían llegar en rachas, pero gracias a Kafka podemos escribir la aplicación de procesado del stream sin tener en cuenta esas consideraciones.

In [1]:
import pandas as pd
from confluent_kafka import Producer
import time
import json
import random
import uuid

from ejercicios.votes import SEED, TOPIC_VOTES, DATA
random.seed(SEED)

In [2]:
ds = pd.read_csv(DATA)
ds.head()

,Codigo,Partido,Votos
0,4001,Blanco,5
1,4001,Nulos,9
2,4001,PP,267
3,4001,PSOE,356
4,4001,PODEMOS_IU_EQUO,65


In [3]:
def choose_partition(num_partitions, key):
    if num_partitions == 1 or key is None:
        return 0
    else:
        try:
            as_num = int(key[0], base=16)
            return as_num % num_partitions
        except:
            return 0


In [4]:
ds['Votos'].sum()

24157982

In [5]:
p = Producer({'bootstrap.servers': 'localhost:9092'})
m = None
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        pass

while True:
    # Trigger any available delivery report callbacks from previous produce() calls
    p.poll(0)
    topic_list = p.list_topics(TOPIC_VOTES)
    topic_metadata = topic_list.topics[TOPIC_VOTES]
    num_partitions = len(topic_metadata.partitions.keys())
    
    vote_id = str(uuid.uuid4())

    if ds.size == 0:
        print('All votes sent!')
        break

    vote = ds.sample(n=1)
    m = vote
    as_dict = vote.to_dict('records')[0]
    num_votos = as_dict['Votos']
    as_dict.pop('Votos')

    p.produce(TOPIC_VOTES,
              key=vote_id,
              partition=choose_partition(num_partitions, vote_id),
              value=json.dumps(as_dict).encode('utf-8'),
              callback=delivery_report)
    
    if num_votos == 1:
        ds = ds.drop(index=vote.index, axis=0)
        print('Removing {}'.format(as_dict))
    else:
        ds.at[vote.index, 'Votos'] = num_votos - 1

    time.sleep(1)

p.flush()



Removing {'Codigo': 16206, 'Partido': 'VOX'}
Removing {'Codigo': 44160, 'Partido': 'EB'}
Removing {'Codigo': 22075, 'Partido': 'PACMA'}
Removing {'Codigo': 46246, 'Partido': 'P_LIB'}
Removing {'Codigo': 39076, 'Partido': 'SAIn'}
Removing {'Codigo': 32057, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 10217, 'Partido': 'Blanco'}
Removing {'Codigo': 23090, 'Partido': 'PCPE'}
Removing {'Codigo': 5066, 'Partido': 'UPyD'}
Removing {'Codigo': 21011, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 46014, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 19242, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 42017, 'Partido': 'Nulos'}
Removing {'Codigo': 31217, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 46012, 'Partido': 'UPyD'}
Removing {'Codigo': 17039, 'Partido': 'PCPE'}
Removing {'Codigo': 18030, 'Partido': 'UPyD'}
Removing {'Codigo': 49110, 'Partido': 'UPyD'}
Removing {'Codigo': 41101, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}

Removing {'Codigo': 19110, 'Partido': 'PCOE'}
Removing {'Codigo': 22023, 'Partido': 'PCPE'}
Removing {'Codigo': 26164, 'Partido': 'UPyD'}
Removing {'Codigo': 6100, 'Partido': 'Blanco'}
Removing {'Codigo': 47164, 'Partido': 'VOX'}
Removing {'Codigo': 34180, 'Partido': 'Blanco'}
Removing {'Codigo': 28114, 'Partido': 'Nulos'}
Removing {'Codigo': 9014, 'Partido': 'IMC'}
Removing {'Codigo': 18036, 'Partido': 'IZAR'}
Removing {'Codigo': 34107, 'Partido': 'Nulos'}
Removing {'Codigo': 23031, 'Partido': 'PCPE'}
Removing {'Codigo': 49154, 'Partido': 'Nulos'}
Removing {'Codigo': 49238, 'Partido': 'Blanco'}
Removing {'Codigo': 37254, 'Partido': 'Nulos'}
Removing {'Codigo': 9246, 'Partido': 'VOX'}
Removing {'Codigo': 50225, 'Partido': 'UPyD'}
Removing {'Codigo': 8002, 'Partido': 'PSOE'}
Removing {'Codigo': 18171, 'Partido': 'VOX'}
Removing {'Codigo': 46115, 'Partido': 'CCD'}
Removing {'Codigo': 47123, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 3017, 'Partido': 'RECORTES_CERO_GRUPO_

Removing {'Codigo': 34201, 'Partido': 'Blanco'}
Removing {'Codigo': 29062, 'Partido': 'AND'}
Removing {'Codigo': 45047, 'Partido': 'UPyD'}
Removing {'Codigo': 40204, 'Partido': 'Blanco'}
Removing {'Codigo': 22064, 'Partido': 'UPyD'}
Removing {'Codigo': 49053, 'Partido': 'PREPAL'}
Removing {'Codigo': 47229, 'Partido': 'Nulos'}
Removing {'Codigo': 2036, 'Partido': 'Cs'}
Removing {'Codigo': 46124, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 32012, 'Partido': 'BNG_NS'}
Removing {'Codigo': 9224, 'Partido': 'Blanco'}
Removing {'Codigo': 3134, 'Partido': 'SOMVAL'}
Removing {'Codigo': 22012, 'Partido': 'Nulos'}
Removing {'Codigo': 47035, 'Partido': 'VOX'}
Removing {'Codigo': 37201, 'Partido': 'Nulos'}
Removing {'Codigo': 29073, 'Partido': 'PCPE'}
Removing {'Codigo': 49102, 'Partido': 'UPyD'}
Removing {'Codigo': 5049, 'Partido': 'Blanco'}
Removing {'Codigo': 24039, 'Partido': 'UPyD'}
Removing {'Codigo': 46132, 'Partido': 'P_LIB'}
Removing {'Codigo': 19167, 'Partido': 'FE_de_las_

Removing {'Codigo': 29028, 'Partido': 'UPyD'}
Removing {'Codigo': 34049, 'Partido': 'FE_de_las_JONS'}
Removing {'Codigo': 18907, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 49013, 'Partido': 'Blanco'}
Removing {'Codigo': 36901, 'Partido': 'PCPE'}
Removing {'Codigo': 34196, 'Partido': 'PCPE'}
Removing {'Codigo': 34903, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 29065, 'Partido': 'Nulos'}
Removing {'Codigo': 20023, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 39004, 'Partido': 'EB'}
Removing {'Codigo': 24901, 'Partido': 'UPyD'}
Removing {'Codigo': 31095, 'Partido': 'SAIn'}
Removing {'Codigo': 3022, 'Partido': 'UPyD'}
Removing {'Codigo': 33067, 'Partido': 'VOX'}
Removing {'Codigo': 34038, 'Partido': 'Nulos'}
Removing {'Codigo': 24203, 'Partido': 'Nulos'}
Removing {'Codigo': 3086, 'Partido': 'Blanco'}
Removing {'Codigo': 44258, 'Partido': 'EB'}
Removing {'Codigo': 19259, 'Partido': 'Blanco'}
Removing {'Codigo': 5185, 'Partido': 'Nulos'}
Remov

Removing {'Codigo': 40170, 'Partido': 'VOX'}
Removing {'Codigo': 42194, 'Partido': 'FE_de_las_JONS'}
Removing {'Codigo': 47146, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 21076, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 5249, 'Partido': 'Nulos'}
Removing {'Codigo': 6017, 'Partido': 'Nulos'}
Removing {'Codigo': 21043, 'Partido': 'PCPE'}
Removing {'Codigo': 47200, 'Partido': 'VOX'}
Removing {'Codigo': 46016, 'Partido': 'P_LIB'}
Removing {'Codigo': 34068, 'Partido': 'PSOE'}
Removing {'Codigo': 6030, 'Partido': 'PACMA'}
Removing {'Codigo': 40010, 'Partido': 'Nulos'}
Removing {'Codigo': 50045, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 37151, 'Partido': 'PODEMOS_IU_EQUO'}
Removing {'Codigo': 31101, 'Partido': 'Ln'}
Removing {'Codigo': 46162, 'Partido': 'UPyD'}
Removing {'Codigo': 27062, 'Partido': 'Blanco'}
Removing {'Codigo': 19248, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 9025, 'Partido': 'PACMA'}
Removing {'Codigo': 3

Removing {'Codigo': 50296, 'Partido': 'MAS'}
Removing {'Codigo': 43130, 'Partido': 'Blanco'}
Removing {'Codigo': 22047, 'Partido': 'ENTABAN'}
Removing {'Codigo': 22009, 'Partido': 'VOX'}
Removing {'Codigo': 44206, 'Partido': 'PCPE'}
Removing {'Codigo': 40115, 'Partido': 'Nulos'}
Removing {'Codigo': 37091, 'Partido': 'Nulos'}
Removing {'Codigo': 45055, 'Partido': 'FE_de_las_JONS'}
Removing {'Codigo': 45125, 'Partido': 'FE_de_las_JONS'}
Removing {'Codigo': 22225, 'Partido': 'VOX'}
Removing {'Codigo': 8222, 'Partido': 'RECORTES_CERO_GRUPO_VERDE'}
Removing {'Codigo': 25909, 'Partido': 'PACMA'}
Removing {'Codigo': 9135, 'Partido': 'IMC'}
Removing {'Codigo': 4007, 'Partido': 'Blanco'}
Removing {'Codigo': 6074, 'Partido': 'VOX'}
Removing {'Codigo': 35003, 'Partido': 'SAIn'}
Removing {'Codigo': 19041, 'Partido': 'UPyD'}
Removing {'Codigo': 42113, 'Partido': 'Blanco'}
Removing {'Codigo': 47015, 'Partido': 'PACMA'}
Removing {'Codigo': 15057, 'Partido': 'SAIn'}
Removing {'Codigo': 18024, 'Partido

KeyboardInterrupt: 

In [56]:
ds['Votos'].sum()

24154452